In [1]:
# import Pandas
import pandas as pd

# import os
import os

In [9]:
# Input data
# 1) NBA Player Stats since 1950
# https://www.kaggle.com/drgilermo/nba-players-stats
# Flat file (CSV) dataset from Kaggle
player_stats_inp = pd.read_csv(os.path.join("..", "RawData", "Seasons_Stats.csv"))

# 2) NBA Play Salaries 1999-2019
# http://www.espn.com/nba/salaries
# Structured dataset scrapped from web
# See PreScripts/web_scrape_espn.ipynb for the scraping process
player_salary_inp = pd.read_csv(os.path.join("..", "RawData", "player_salary.csv"))

In [19]:
# Clean up dataset 1 to get ready for merging
# keep useful columns
player_stats = player_stats_inp[["Year", "Player", "Pos", "Age", "Tm", "FG%", "eFG%", "FT%", "PTS"]]

# Get rid of NaN values
player_stats = player_stats.dropna()

# Make the values that should be integer integer
player_stats.Year = player_stats.Year.astype(int)
player_stats.Age = player_stats.Age.astype(int)

# rname column
player_stats = player_stats.rename(columns={"Pos": "Position"})

player_stats.tail()

,Year,Player,Position,Age,Tm,FG%,eFG%,FT%,PTS
24686,2017,Cody Zeller,PF,24,CHO,0.571,0.571,0.679,639.0
24687,2017,Tyler Zeller,C,27,BOS,0.494,0.494,0.564,178.0
24688,2017,Stephen Zimmerman,C,20,ORL,0.323,0.323,0.600,23.0
24689,2017,Paul Zipser,SF,22,CHI,0.398,0.473,0.775,240.0
24690,2017,Ivica Zubac,C,19,LAL,0.529,0.529,0.653,284.0


In [17]:
# Clean up dataset 2 to get ready for merging
player_salary = player_salary_inp[["Name", "Position", "Team", "Salary", "Year"]]


player_salary.tail()

,Name,Position,Team,Salary,Year
7505,Al Harrington,PF,Indiana Pacers,"$745,000",2000
7506,Tim James,F,Miami Heat,"$728,000",2000
7507,Jumaine Jones,SF,Philadelphia 76ers,"$684,000",2000
7508,Scott Padgett,PF,Utah Jazz,"$679,000",2000
7509,Rafer Alston,PG,Milwaukee Bucks,"$301,000",2000
